# Coffee Quality Prediction ☕️☕️☕️

🗂️ Dataset: [Link](https://www.kaggle.com/datasets/fatihb/coffee-quality-data-cqi/data)

🚀 Tasks: 
1. Conduct an Exploratory Data Analysis (EDA) on the given dataset. Identify any features that might influence coffee quality or uncover other noteworthy insights from the data. Start form [Here](#overview)
2. Discuss whether any data cleansing is required before feeding the data into the model. Explain the necessary steps for data cleansing and the rationale behind them. [Here](#task_2)
3. Develop a machine learning to predict coffee quality. [Here](#model_prediction)
4. Justify your choice of model in the previous question, detailing why it is suitable for this task. [Here](#conclusion)

📑 Contents:
- [Overview](#Overview)
- [Understanding Dataset](#understanding_dataset)
- [Data Quality](#data_quality)
    - [Data Type](##data_type)
    - [Filling Missing Values](##filling_missing_values)
    - [Remove Irrevant Columns](##remove_irrevant_columns)
- [Exploring Data](#exploring_data)
    - [Numerical Data](##numerical_data)
    - [Categorical Data](##categorical_data)
- [Model Prediction](#model_prediction)
- [Conclusion](#conclusion)

# Overview

The dataset used in this project is the Coffee Quality Data from the Coffee Quality Institute (CQI). It contains detailed information about coffee metadata. I separated dataset into 4 main groups based on [Link](https://github.com/jldbc/coffee-quality-database)

`📊 Quality Measures`
- Aroma
- Flavor
- Aftertaste
- Acidity
- Body
- Balance
- Uniformity
- Clean Cup
- Sweetness
- Overall
- Total Cup Points - Calculated from 10 features above. 
- Defects
- Moisture Percentage
- Category One Defects
- Quakers
- Category Two Defects

`🌱 Bean Metadata`
- Harvest Year - has 2 formats 'YYYY-YYYY' and 'YYYY'
- Variety
- Processing Method
- Color


`👨🏻‍🌾 Farm and export Metadata`
- Country of Origin
- Farm Name
- Lot Number
- Mill - In this dataset, mill likely to refer to location name that process coffee more than type of mill. 
- ICO Number - used as a traceability code of coffee product from origin to customer [Link](http://www.ico.org/documents/icc-102-9e-rules-certificates-final.pdf)
- Company
- Altitude
- Region
- Producer
- Number of Bags
- Bag Weight
- In-Country Partner
- Owner

`🏅 Certification Metadata`
- Grading Date
- Expiration - In this dataset, I found that the period between expriation and grading date is 12 months. so I assume this data is exporation of grading score.
- Certification Body
- Certification Address
- Certification Contact
- Status

In this project, I want to create a prediction model for coffee quality 'Total Cup Points' which I will explore and analysis there is any features (except 10 features) have significant to be used in prediction or not. Note: we're already know exact function to compute 'Total Cup Points' using 10 quality features.

<a id="task_2"></a>
🚀 Task 2 answer:

Data cleansing process is necessary process to ensure that the resulting models are accurate, reliable, and effective. These are steps:
1. Understanding Dataset: understand the dataset structure with information to consider this dataset has potential to be used or not.
2. Data Quality: 
    2.1 Data Type: correct the data in dataset to appropriate datatype
    2.2 filling missing value: instead of dropping incomplete data. The data can be filled with appropriate method.
    2.3 Remove Irrevant Columns: from dataset infomation remove irrevant column that may misleading prediction model and reduce risk of overfitting.
3. Exploring Data:
    3.1 Numerical Data: Remove outlier because they can misleading analysis and inaccurate prediction.
    3.2 Categorical Data: Leveling / Grouping the data value that has their minority less than threshold to improve the overll data characteristic.

<a id="understanding_dataset"></a>
# Understanding Dataset

In [1]:
# import libraries
import json
import numpy as np
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import re

# set pandas options to display maximum columns
pd.set_option('display.max_columns', 100)

In [2]:
# 1. import dataset
filepath = 'data/df_arabica_clean.csv'
raw_df = pd.read_csv(filepath)
print(f"> Original data shape: {raw_df.shape} | rows: {raw_df.shape[0]} | columns: {raw_df.shape[1]}")

> Original data shape: (207, 41) | rows: 207 | columns: 41


In [3]:
# see an example of the data
raw_df.head(2)

,Unnamed: 0,ID,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,Producer,Number of Bags,Bag Weight,In-Country Partner,Harvest Year,Grading Date,Owner,Variety,Status,Processing Method,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean Cup,Sweetness,Overall,Defects,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Color,Category Two Defects,Expiration,Certification Body,Certification Address,Certification Contact
0,0,0,Colombia,Finca El Paraiso,CQU2022015,Finca El Paraiso,NaN,Coffee Quality Union,1700-1930,"Piendamo,Cauca",Diego Samuel Bermudez,1,35 kg,Japan Coffee Exchange,2021 / 2022,"September 21st, 2022",Coffee Quality Union,Castillo,Completed,Double Anaerobic Washed,8.58,8.5,8.42,8.58,8.25,8.42,10.0,10.0,10.0,8.58,0.0,89.33,11.8,0,0,green,3,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
1,1,1,Taiwan,Royal Bean Geisha Estate,"The 2022 Pacific Rim Coffee Summit,T037",Royal Bean Geisha Estate,NaN,Taiwan Coffee Laboratory,1200,Chiayi,曾福森,1,80 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Gesha,Completed,Washed / Wet,8.50,8.5,7.92,8.00,7.92,8.25,10.0,10.0,10.0,8.50,0.0,87.58,10.5,0,0,blue-green,0,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"


In [4]:
# 2. rename columns for better readability
columns = list(raw_df.columns)
rename_columns = {column:re.sub(r'\s+', '_', column.lower()) for column in columns}
# In-Country Partner column has a special character, I will replace it with '_'
rename_columns['In-Country Partner'] = 'in_country_partner'
# rename the columns
raw_df.rename(columns=rename_columns, inplace=True)
raw_df.head(2)

,unnamed:_0,id,country_of_origin,farm_name,lot_number,mill,ico_number,company,altitude,region,producer,number_of_bags,bag_weight,in_country_partner,harvest_year,grading_date,owner,variety,status,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,overall,defects,total_cup_points,moisture_percentage,category_one_defects,quakers,color,category_two_defects,expiration,certification_body,certification_address,certification_contact
0,0,0,Colombia,Finca El Paraiso,CQU2022015,Finca El Paraiso,NaN,Coffee Quality Union,1700-1930,"Piendamo,Cauca",Diego Samuel Bermudez,1,35 kg,Japan Coffee Exchange,2021 / 2022,"September 21st, 2022",Coffee Quality Union,Castillo,Completed,Double Anaerobic Washed,8.58,8.5,8.42,8.58,8.25,8.42,10.0,10.0,10.0,8.58,0.0,89.33,11.8,0,0,green,3,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
1,1,1,Taiwan,Royal Bean Geisha Estate,"The 2022 Pacific Rim Coffee Summit,T037",Royal Bean Geisha Estate,NaN,Taiwan Coffee Laboratory,1200,Chiayi,曾福森,1,80 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Gesha,Completed,Washed / Wet,8.50,8.5,7.92,8.00,7.92,8.25,10.0,10.0,10.0,8.50,0.0,87.58,10.5,0,0,blue-green,0,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"


In [5]:
# 3. get information of the data
def get_info(df: pd.DataFrame):
    """
    This function returns the information of the dataframe
    I separate the data into two dataframes, one for numeric data and the other for object data
    Args:
        df: a dataframe that you want to get the information
    Returns:
        numeric_df: a dataframe that contains the information of numeric data
        object_df: a dataframe that contains the information of object data
    """
    numeric_data = []
    object_data = []
    for column in df.columns:
        series = df[column]
        total = series.count() + series.isnull().sum()
        if series.dtype == object or series.dtype == bool:
            object_data.append({
                'column': column,
                'dtype': series.dtype,
                'valid': series.count(),
                'missing': series.isnull().sum(),
                'unique': series.nunique(),
                'unique_pct': round((series.nunique() *100 / total), 2),
                'top': series.mode().values[0],
                'freq': series.value_counts().values[0]
            })
        else:
            numeric_data.append({
                'column': column,
                'dtype': series.dtype,
                'valid': series.count(),
                'missing': series.isnull().sum(),
                'unique': series.nunique(),
                'unique_pct': round((series.nunique()*100 / total), 2),
                'mean': round(series.mean(), 2),
                'std': round(series.std(), 2),
                'min': series.min(),
                '25%': series.quantile(0.25),
                '50%': series.quantile(0.50),
                '75%': series.quantile(0.75),
                'max': series.max(),
            })
    numeric_df = pd.DataFrame(numeric_data)
    object_df = pd.DataFrame(object_data)
    return numeric_df, object_df

In [6]:
# get the information of the dataframe
raw_numeric_df, raw_object_df = get_info(raw_df)
print(f"> Raw data in numerical type: #{raw_numeric_df.shape[0]}")
print(f"> Raw data in object type: #{raw_object_df.shape[0]}")

> Raw data in numerical type: #19
> Raw data in object type: #22


In [7]:
raw_numeric_df

,column,dtype,valid,missing,unique,unique_pct,mean,std,min,25%,50%,75%,max
0,unnamed:_0,int64,207,0,207,100.00,103.00,59.90,0.00,51.50,103.00,154.500,206.00
1,id,int64,207,0,207,100.00,103.00,59.90,0.00,51.50,103.00,154.500,206.00
2,number_of_bags,int64,207,0,55,26.57,155.45,244.48,1.00,1.00,14.00,275.000,2240.00
3,aroma,float64,207,0,19,9.18,7.72,0.29,6.50,7.58,7.67,7.920,8.58
4,flavor,float64,207,0,19,9.18,7.74,0.28,6.75,7.58,7.75,7.920,8.50
5,aftertaste,float64,207,0,20,9.66,7.60,0.28,6.67,7.42,7.58,7.750,8.42
6,acidity,float64,207,0,19,9.18,7.69,0.26,6.83,7.50,7.67,7.875,8.58
7,body,float64,207,0,17,8.21,7.64,0.23,6.83,7.50,7.67,7.750,8.25
8,balance,float64,207,0,18,8.70,7.64,0.26,6.67,7.50,7.67,7.790,8.42
9,uniformity,float64,207,0,3,1.45,9.99,0.10,8.67,10.00,10.00,10.000,10.00


In [8]:
raw_object_df

,column,dtype,valid,missing,unique,unique_pct,top,freq
0,country_of_origin,object,207,0,22,10.63,Taiwan,61
1,farm_name,object,205,2,172,83.09,Doi Tung Development Project,7
2,lot_number,object,206,1,187,90.34,1,11
3,mill,object,204,3,162,78.26,Dry Mill,11
4,ico_number,object,75,132,67,32.37,non,5
5,company,object,207,0,72,34.78,Taiwan Coffee Laboratory,51
6,altitude,object,206,1,97,46.86,1200,23
7,region,object,205,2,120,57.97,Chiayi,12
8,producer,object,206,1,172,83.09,Doi Tung Development Project,7
9,bag_weight,object,207,0,39,18.84,30 kg,39


In [9]:
# 4. Remove columns from data information
# 4.1 if unique pct is > 80% then I will discard these columns
threshold = 80
unique_numeric_columns = raw_numeric_df[raw_numeric_df['unique_pct'] > threshold]['column'].values
unique_object_columns = raw_object_df[raw_object_df['unique_pct'] > threshold]['column'].values
unique_columns = np.concatenate([unique_numeric_columns, unique_object_columns])
print(f"> Columns to be removed (unique pct is > 80%): {unique_columns}")

# 4.2 columns that have only one unique value
constant_numeric_columns = raw_numeric_df[raw_numeric_df['unique'] == 1]['column'].values
constant_object_columns = raw_object_df[raw_object_df['unique'] == 1]['column'].values
constant_columns = np.concatenate([constant_numeric_columns, constant_object_columns])
print(f"> Columns to be removed (unique value): {constant_columns}")

remove_columns = np.concatenate([unique_columns, constant_columns])
print(f"> Columns to be removed (all): {remove_columns}")

df = raw_df.drop(columns=remove_columns)
print(f"> Data shape after dropping unique columns: {df.shape} | rows: {df.shape[0]} | columns: {df.shape[1]}")

> Columns to be removed (unique pct is > 80%): ['unnamed:_0' 'id' 'farm_name' 'lot_number' 'producer']
> Columns to be removed (unique value): ['clean_cup' 'sweetness' 'defects' 'status']
> Columns to be removed (all): ['unnamed:_0' 'id' 'farm_name' 'lot_number' 'producer' 'clean_cup'
 'sweetness' 'defects' 'status']
> Data shape after dropping unique columns: (207, 32) | rows: 207 | columns: 32


<a id="data_quality"></a>
# Data Quality

<a id="data_type"></a>
## Data Type

> 'altitude', 'bag_weight', 'harvest_year', 'grading_date', 'expiration' from raw_object_df can be converted to numeric data

In [10]:
def clean_altitude(x: str):
    """
    This function cleans the 'altitude' column
    Args:
        x: altitude value in string
    Returns:
        altitude value in integer
    """
    if pd.isnull(x):
        return x
    match = re.match("(\d+)-(\d+)", x)
    if match:
        return (int(match.group(1)) + int(match.group(2))) / 2
    else:
        return int(x)

In [11]:
# 1. altitude
print(f"> Unique values of 'altitude': {df['altitude'].unique()}")
# from the unique values, I found that there are some ranges of altitude, so I will use the average of the range as the value of the altitude
# Clean data  '~', ' - ', ' A ' to '-'
df['altitude'] = df['altitude'].str.replace('~', '-').str.replace(' - ', '-').str.replace(' A ', '-')
df['altitude'] = df['altitude'].apply(clean_altitude)
print(f"> Unique values of 'altitude' after cleaning: {df['altitude'].unique()}, data type: {df['altitude'].dtype}")

> Unique values of 'altitude': ['1700-1930' '1200' '1300' '1900' '1850-2100' '1668' '1250' '1400-1700'
 '1800-2200' '2000' '1900-2000' '1850' '1100' '1900-2100' '1570-1600'
 '850' '1500-1700' '1350' '1060' '600' '150-250' '668' '1950'
 '2150 - 2350' '1700' '650' '1600-1900' '300-500' '1000' '800' '1905'
 '150' '1600' '4700' '350-400' '230' '1750' '1654' '1400' '4895' '200-300'
 '700' '1450' '520' '1368' '1943' '400' '1300-1400' '2361' '1500' '2100'
 '1250-1350' '1500-1600' '1800' '1600-1750' '1411' '4895 A 5650' '400-600'
 nan '200-400' '1470' '250-400' '250-300' '1200 - 1580' '1400 - 1900'
 '1280-1325' '300' '750' '1300-1500' '950' '1100-1200' '1390' '340'
 '1200 - 1300' '1650' '1280' '900' '165' '465' '640' '1340' '139'
 '1200-1350' '1040' '140' '1500-1950' '460' '500' '800-1200' '1350-1550'
 '1200~1600' '5400' '900-1000' '1574' '435' '600-800' '1300-1800'
 '850-1100']
> Unique values of 'altitude' after cleaning: [1815.  1200.  1300.  1900.  1975.  1668.  1250.  1550.  2000.  1950.


In [12]:
# 2. 'bag_weight' column
print(f"> Unique values of 'bag_weight': {df['bag_weight'].unique()}")
# Clean data 'kg', 'Kg' to ''
df['bag_weight'] = df['bag_weight'].str.replace('kg', '').str.replace('Kg', '').str.strip()
df['bag_weight'] = df['bag_weight'].astype(float)
print(f"> Unique values of 'bag_weight' after cleaning: {df['bag_weight'].unique()}, data type: {df['bag_weight'].dtype}")

> Unique values of 'bag_weight': ['35 kg' '80 kg' '25 kg' '22 kg' '24 kg' '30 kg' '27 kg' '90 kg' '60 kg'
 '15 kg' '5 kg' '2 kg' '1 kg' '20 kg' '50 kg' '100 kg' '85 kg' '48 kg'
 '19200 kg' '10 kg' '69 kg' '13 kg' '4 kg' '36 kg' '16 kg' '45 kg'
 '104 kg' '300 kg' '70 kg' '110 kg' '40 kg' '8 kg' '320 kg' '12 kg'
 '32 kg' '200 kg' '3 kg' '6 kg' '59 kg']
> Unique values of 'bag_weight' after cleaning: [3.50e+01 8.00e+01 2.50e+01 2.20e+01 2.40e+01 3.00e+01 2.70e+01 9.00e+01
 6.00e+01 1.50e+01 5.00e+00 2.00e+00 1.00e+00 2.00e+01 5.00e+01 1.00e+02
 8.50e+01 4.80e+01 1.92e+04 1.00e+01 6.90e+01 1.30e+01 4.00e+00 3.60e+01
 1.60e+01 4.50e+01 1.04e+02 3.00e+02 7.00e+01 1.10e+02 4.00e+01 8.00e+00
 3.20e+02 1.20e+01 3.20e+01 2.00e+02 3.00e+00 6.00e+00 5.90e+01], data type: float64


In [13]:
# 3. 'harvest_year' column
print(f"> Unique values of 'harvest_year': {df['harvest_year'].unique()}")
# Clean data if value is range then get the first year
df['harvest_year'] = df['harvest_year'].str.extract(r'(\d{4})')
df['harvest_year'] = df['harvest_year'].astype(int)
print(f"> Unique values of 'harvest_year' after cleaning: {df['harvest_year'].unique()}, data type: {df['harvest_year'].dtype}")

> Unique values of 'harvest_year': ['2021 / 2022' '2022' '2022 / 2023' '2021' '2017 / 2018' '2018 / 2019'
 '2023']
> Unique values of 'harvest_year' after cleaning: [2021 2022 2017 2018 2023], data type: int64


In [14]:
# 4. 'grading_date' column to string datetime format
# Create a function to remove ordinal suffixes
def remove_ordinals(date_str):
    return re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)

# grading_date is not in datetime format
print(df['grading_date'].head())
df['grading_date'] = pd.to_datetime(
    df['grading_date'].apply(remove_ordinals),
    format="%B %d, %Y"
).astype(str)
print("\nAfter converting to datetime format:")
print(df['grading_date'].head())

0    September 21st, 2022
1     November 15th, 2022
2     November 15th, 2022
3    September 21st, 2022
4         March 6th, 2023
Name: grading_date, dtype: object

After converting to datetime format:
0    2022-09-21
1    2022-11-15
2    2022-11-15
3    2022-09-21
4    2023-03-06
Name: grading_date, dtype: object


In [15]:
# 5. 'expiration' column to datetime
# grading_date is not in datetime format
print(df['expiration'].head())
df['expiration'] = pd.to_datetime(
    df['expiration'].apply(remove_ordinals),
    format="%B %d, %Y"
).astype(str)
print("\nAfter converting to datetime format:")
print(df['expiration'].head())

0    September 21st, 2023
1     November 15th, 2023
2     November 15th, 2023
3    September 21st, 2023
4         March 5th, 2024
Name: expiration, dtype: object

After converting to datetime format:
0    2023-09-21
1    2023-11-15
2    2023-11-15
3    2023-09-21
4    2024-03-05
Name: expiration, dtype: object


In [16]:
numeric_df, object_df = get_info(df)
print(f"> Data in numerical type: #{numeric_df.shape[0]}")
print(f"> Data in object type: #{object_df.shape[0]}")

> Data in numerical type: #17
> Data in object type: #15


In [17]:
numeric_df

,column,dtype,valid,missing,unique,unique_pct,mean,std,min,25%,50%,75%,max
0,altitude,float64,206,1,76,36.71,1311.74,721.50,139.00,1010.00,1321.25,1600.000,5400.00
1,number_of_bags,int64,207,0,55,26.57,155.45,244.48,1.00,1.00,14.00,275.000,2240.00
2,bag_weight,float64,207,0,39,18.84,227.78,1878.91,1.00,15.00,30.00,60.000,19200.00
3,harvest_year,int64,207,0,5,2.42,2021.43,0.66,2017.00,2021.00,2021.00,2022.000,2023.00
4,aroma,float64,207,0,19,9.18,7.72,0.29,6.50,7.58,7.67,7.920,8.58
5,flavor,float64,207,0,19,9.18,7.74,0.28,6.75,7.58,7.75,7.920,8.50
6,aftertaste,float64,207,0,20,9.66,7.60,0.28,6.67,7.42,7.58,7.750,8.42
7,acidity,float64,207,0,19,9.18,7.69,0.26,6.83,7.50,7.67,7.875,8.58
8,body,float64,207,0,17,8.21,7.64,0.23,6.83,7.50,7.67,7.750,8.25
9,balance,float64,207,0,18,8.70,7.64,0.26,6.67,7.50,7.67,7.790,8.42


<a id="filling_missing_values"></a>
## Filling Missing Values

In [18]:
# check missing values
missing_df = numeric_df.query("missing > 0")
# concat to the missing values of object_df
missing_df = pd.concat([missing_df, object_df.query("missing > 0")], axis=0)
missing_df[['column', 'dtype', 'missing', 'unique', 'unique_pct']].sort_values(by='missing', ascending=False)

,column,dtype,missing,unique,unique_pct
2,ico_number,object,132,67,32.37
8,variety,object,6,48,23.19
9,processing_method,object,5,10,4.83
1,mill,object,3,162,78.26
4,region,object,2,120,57.97
0,altitude,float64,1,76,36.71


1. 'ico_number'
> To get 'ico_number', the coffee must meet their export quality standards (P16 in the [link](http://www.ico.org/documents/icc-102-9e-rules-certificates-final.pdf)). I decide to transform it to boolean datatype.

In [19]:
print(df['ico_number'].unique())

[nan '033/DE/503/002 and 033/DE/268/002' '010/0296/600' '010/0475/0207'
 'non' '033/DE/163/001' '010/1081' 'ICO: 010/ 0218/0065' '010/0891/00041'
 '11/35' '11/15/118' '11/951/279' '11/441/50' '0033/0004' '002-1894-0006'
 '2-237-05' '11/15/117' '11/54876/03' '5-0025-0131' '13-111-027'
 '010/0148/0712' '11-63-657' '010/ 0218/0065' '11/15/96' '13-111-030'
 '11/441/49' '11/15/51' '11/15/59' '033/DE/491/001' '010-0424-0586'
 '033/0010/022' '5-0025-0132' '11/15/95' '3/37/1370' '11/63/908'
 '11/54876/01' '010/0145/0064' '3-0068-00886' '016-2028-0312' '13-167-471'
 '3-0104-00202' '5-0025-0130' '033/0010/032' '13-167-471-2' '13-111-059'
 '13-63-119' '3-59-0654' '3093' '13-167-181' '017-053-0017' '3/01/12857'
 '13-63-118' '9-392-35' '29-169-003' '002/1643/0049' '13-111-057'
 '09-392-144' '13-111-056' '5-0015-0228' '#327' '301R' '11/15/179'
 '002/1738/0038' '017/411' '002/1738/0029' '017-053-0155'
 '105/3/VL7285-005' '002/1208/1016']


In [20]:
# from unique value, one of 'ico_number' is 'non' which is not a valid value. I will replace it with np.nan
df['ico_number'] = df['ico_number'].replace('non', np.nan)

# Convert 'ico_number' to boolean
df['ico_number'] = df['ico_number'].notnull()
df['ico_number'].value_counts(normalize=True)

ico_number
False    0.661836
True     0.338164
Name: proportion, dtype: float64

2. 'altitude'

In [21]:
print(df['altitude'].unique())

[1815.  1200.  1300.  1900.  1975.  1668.  1250.  1550.  2000.  1950.
 1850.  1100.  1585.   850.  1600.  1350.  1060.   600.   200.   668.
 2250.  1700.   650.  1750.   400.  1000.   800.  1905.   150.  4700.
  375.   230.  1654.  1400.  4895.   250.   700.  1450.   520.  1368.
 1943.  2361.  1500.  2100.  1800.  1675.  1411.  5272.5  500.     nan
  300.  1470.   325.   275.  1390.  1650.  1302.5  750.   950.  1150.
  340.  1280.   900.   165.   465.   640.  1340.   139.  1275.  1040.
  140.  1725.   460.  5400.  1574.   435.   975. ]


In [22]:
df[df['altitude'].isnull()]

,country_of_origin,mill,ico_number,company,altitude,region,number_of_bags,bag_weight,in_country_partner,harvest_year,grading_date,owner,variety,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,overall,total_cup_points,moisture_percentage,category_one_defects,quakers,color,category_two_defects,expiration,certification_body,certification_address,certification_contact
105,Colombia,NaN,False,Coffee Quality Institute,NaN,NaN,1,1.0,Barista and Coffee Academy of Asia,2022,2022-09-26,Coffee Quality Institute,NaN,NaN,7.83,7.75,7.5,7.58,7.67,7.67,10.0,7.67,83.67,12.4,1,0,greenish,9,2023-09-26,Barista and Coffee Academy of Asia,"The Place Bldg., Tunasan, Muntinlupa City, Met...",Rosario Cruz - +63 995 344 5688


In [23]:
# altitude depends on the region, but the region is not available in the data above. I will use the country to fill the missing values
# df['altitude'] = df.groupby('region')['altitude'].transform(lambda x: x.fillna(x.median()))
df['altitude'] = df.groupby('country_of_origin')['altitude'].transform(lambda x: x.fillna(x.median()))
df.loc[105, 'altitude']

np.float64(1625.0)

3. 'region'
> In this value, I decide to filling missing value of 'region' with 'country_of_origin' and 'altitude'.
If missing 'region' has no match 'altitude', the median of 'region' in the same country is filled.

In [24]:
print(df['region'].unique())

['Piendamo,Cauca' 'Chiayi' 'Laos Borofen Plateau' 'Los Santos,Tarrazu'
 'Popayan,Cauca' 'Chimaltenango' 'KILIMANJARO' 'Guji' 'Acatenango'
 'Yunlin' 'tolima' 'Gedeb,Yirgacheffe,Sidamo'
 'Shibi, Gukeng Township, Yunlin County 郵遞區號 , Taiwan (R.O.C.)'
 'Gukeng Township, Yunlin County' 'Arusha'
 'Guatemala, Fraijanes, Santa Rosa' '卓溪鄉Zhuoxi Township' 'Chiang Mai'
 'Quindio' 'Região Vulcânica' 'Kona' '壽豐鄉Shoufeng Township'
 'Dongshan Dist., Tainan City' 'Oromia' 'Southern Ethiopia Guji' 'OROMIA'
 'Central' 'Caoling , Gukeng Township, Yunlin County'
 '秀林鄉Show Linxia Township' '台灣屏東' '苗栗縣' 'Rwenzori' 'Antigua' 'Santa Rosa'
 'quiche' '新竹縣' 'Aceh Tengah' 'Villa Rica' 'Mbeya' 'Nantou'
 'Campo das Vertentes' 'Boquete' 'Huehuetenango' 'ANTIGUA GUATEMALA'
 'Tarrazu' '( Dongshan Dist., Tainan City)' 'ESTELI' 'Quang Tri'
 'Centro, Lagunetillas-Ajuterique, Comayagua' 'Ethiopia'
 '玉里鎮Yuli Township' "Ka'u district of Big Island" 'Addis Ababa'
 'Chalatenango' 'Sierra Nevada de Santa Marta'
 'Lintong Nihut

In [25]:
non_region_df = df[df['region'].isnull()]
# filter the region with country_of_origin and altitude to fill the missing values
non_region_df[['country_of_origin', 'altitude']].value_counts()

country_of_origin  altitude
Colombia           1625.0      1
Nicaragua          1100.0      1
Name: count, dtype: int64

In [26]:
# fill missing values in 'region' 
for index, row in non_region_df.iterrows():
    country = row['country_of_origin']
    altitude = row['altitude']
    if pd.isnull(country):
        continue
    region = df[(df['country_of_origin'] == country) & (df['altitude'] == altitude)]['region'].mode().values
    if len(region) > 0:
        print(f"Fill the missing value in 'region' with {region[0]} where 'country_of_origin' is {country} and 'altitude' is {region}")
        df.loc[index, 'region'] = region[0]
    else:
        region = df[(df['country_of_origin'] == country)]['region'].mode().values
        if len(region) > 0:
            print(f"Fill the missing value in 'region' with {region[0]} where 'country_of_origin' is {country}")
            df.loc[index, 'region'] = region[0]

Fill the missing value in 'region' with Quindio where 'country_of_origin' is Colombia
Fill the missing value in 'region' with ESTELI where 'country_of_origin' is Nicaragua


In [27]:
# check after filling the missing values
df.loc[non_region_df.index, 'region']

105    Quindio
196     ESTELI
Name: region, dtype: object

4. 'variety'
> In this value, I decide to filling missing value of 'variety' with 'country_of_origin' and 'region'.
If missing 'variety' has no match 'region', the median of 'variety' in the same country is filled.

In [28]:
print(df['variety'].unique())

['Castillo' 'Gesha' 'Java' 'Red Bourbon' 'Sl34+Gesha' 'SL34' 'Bourbon'
 'Ethiopian Heirlooms' 'Caturra' 'Wolishalo,Kurume,Dega' 'Typica'
 'Catimor' 'Castillo Paraguaycito' nan 'SL28' 'SL14' 'Catuai'
 'Yellow Bourbon' 'Catrenic' 'unknown' 'Pacamara'
 'Castillo and Colombia blend' 'Jember,TIM-TIM,Ateng'
 'BOURBON, CATURRA Y CATIMOR' 'Bourbon Sidra' 'Sarchimor'
 'Catimor,Catuai,Caturra,Bourbon' 'Parainema' 'SHG' 'Typica + SL34'
 'MARSELLESA, CATUAI, CATURRA & MARSELLESA, ANACAFE 14, CATUAI'
 'Mundo Novo' 'Red Bourbon,Caturra' 'Lempira' 'Typica Gesha' 'Gayo'
 'Bourbon, Catimor, Caturra, Typica' 'unknow' 'Maragogype'
 'Caturra-Catuai' 'SL28,SL34,Ruiru11' 'Yellow Catuai' 'Catucai'
 'Santander' 'Typica Bourbon Caturra Catimor' 'Caturra,Colombia,Castillo'
 'Castillo,Caturra,Bourbon' 'Pacas' 'Catuai and Mundo Novo']


In [29]:
# from unique values there are some values represent nan values ['unknown', 'unknow'], I will replace them with np.nan
df['variety'] = df['variety'].replace(['unknown', 'unknow'], np.nan)

In [30]:
non_variety_df = df[df['variety'].isnull()]
# filter the variety with country_of_origin and region to fill the missing values
non_variety_df[['country_of_origin', 'region']].value_counts()

country_of_origin  region              
Taiwan             苗栗縣                     5
                   新竹縣                     4
                   桃園市                     2
Brazil             Alta Mogiana-Ibiraci    1
                   Região Vulcânica        1
Colombia           Pereira                 1
                   Quindio                 1
Guatemala          Oriente                 1
Indonesia          Sumatra                 1
Taiwan             新北市                     1
                   臺北市                     1
Name: count, dtype: int64

In [31]:
# fill missing values in 'variety' 
for index, row in non_variety_df.iterrows():
    country = row['country_of_origin']
    region = row['region']
    if pd.isnull(country):
        continue
    variety = df[(df['country_of_origin'] == country) & (df['region'] == region)]['variety'].mode().values
    if len(variety) > 0:
        print(f"Fill the missing value in 'variety' with {variety[0]} where 'country_of_origin' is {country} and 'region' is {region}")
        df.loc[index, 'variety'] = variety[0]
    else:
        variety = df[(df['country_of_origin'] == country)]['variety'].mode().values
        if len(variety) > 0:
            print(f"Fill the missing value in 'variety' with {variety[0]} where 'country_of_origin' is {country}")
            df.loc[index, 'variety'] = variety[0]

Fill the missing value in 'variety' with Mundo Novo where 'country_of_origin' is Brazil
Fill the missing value in 'variety' with Caturra where 'country_of_origin' is Taiwan and 'region' is 苗栗縣
Fill the missing value in 'variety' with Typica where 'country_of_origin' is Taiwan and 'region' is 新北市
Fill the missing value in 'variety' with Caturra where 'country_of_origin' is Colombia
Fill the missing value in 'variety' with Castillo Paraguaycito where 'country_of_origin' is Colombia and 'region' is Quindio
Fill the missing value in 'variety' with Typica where 'country_of_origin' is Taiwan and 'region' is 新竹縣
Fill the missing value in 'variety' with Typica where 'country_of_origin' is Taiwan and 'region' is 新竹縣
Fill the missing value in 'variety' with Caturra where 'country_of_origin' is Taiwan and 'region' is 苗栗縣
Fill the missing value in 'variety' with Caturra where 'country_of_origin' is Taiwan and 'region' is 苗栗縣
Fill the missing value in 'variety' with Caturra where 'country_of_origin

In [32]:
# check after filling the missing values
df.loc[non_variety_df.index, 'variety']

25                Mundo Novo
69                   Caturra
84                    Typica
97                   Caturra
105    Castillo Paraguaycito
108                   Typica
110                   Typica
130                  Caturra
135                  Caturra
153                  Caturra
162                   Typica
167                   Typica
172                   Typica
173                  Caturra
174                   Typica
177                  Caturra
178                  Catimor
189                   Typica
194    Catuai and Mundo Novo
Name: variety, dtype: object

5. 'processing_method'
> In this value, I decide to filling missing value of 'processing_method' with 'country_of_origin' and 'variety'.

In [33]:
print("Unique Values:", df['processing_method'].unique())

Unique Values: ['Double Anaerobic Washed' 'Washed / Wet' 'Semi Washed' 'Honey,Mossto'
 'Natural / Dry' 'Pulped natural / honey' nan
 'Double Carbonic Maceration / Natural' 'Wet Hulling' 'Anaerobico 1000h'
 'SEMI-LAVADO']


In [34]:
# fill missing values in 'processing_method' with the condition of country_of_origin and variety
non_processing_method_df = df[df['processing_method'].isnull()]
non_processing_method_df[['country_of_origin', 'variety']].value_counts()

country_of_origin  variety              
Colombia           Castillo Paraguaycito    2
Indonesia          Catimor                  1
Taiwan             Gesha                    1
                   Typica                   1
Name: count, dtype: int64

In [35]:
for index, row in non_processing_method_df.iterrows():
    country = row['country_of_origin']
    variety = row['variety']
    if pd.isnull(country):
        continue
    processing_method = df[(df['country_of_origin'] == country) & (df['variety'] == variety)]['processing_method'].mode().values
    if len(processing_method) > 0:
        print(f"Fill the missing value in 'processing_method' with {processing_method[0]} where 'country_of_origin' is {country} and 'variety' is {variety}")
        df.loc[index, 'processing_method'] = processing_method[0]
    else:
        variety = df[(df['country_of_origin'] == country)]['processing_method'].mode().values
        if len(variety) > 0:
            print(f"Fill the missing value in 'processing_method' with {variety[0]} where 'country_of_origin' is {country}")
            df.loc[index, 'processing_method'] = variety[0]

Fill the missing value in 'processing_method' with Washed / Wet where 'country_of_origin' is Colombia
Fill the missing value in 'processing_method' with Pulped natural / honey where 'country_of_origin' is Taiwan and 'variety' is Typica
Fill the missing value in 'processing_method' with Pulped natural / honey where 'country_of_origin' is Indonesia and 'variety' is Catimor
Fill the missing value in 'processing_method' with Washed / Wet where 'country_of_origin' is Colombia and 'variety' is Castillo Paraguaycito
Fill the missing value in 'processing_method' with Washed / Wet where 'country_of_origin' is Taiwan and 'variety' is Gesha


In [36]:
# check after filling the missing values
df.loc[non_processing_method_df.index, 'processing_method']

23               Washed / Wet
44     Pulped natural / honey
51     Pulped natural / honey
105              Washed / Wet
143              Washed / Wet
Name: processing_method, dtype: object

6. 'mill'
From my search 'mill' refers to facilities where coffee is processed. There are 2 main types

    - Wet mill
    - Dry mill

> mill is the location where processing methods are applied. In this dataset found that 'mill' is likely to refer to location name more than type of mill which I thought that is quite specific with unique_pct ~80%.Moreover most of 'mill' data is not in english which made hard to classify mill type so I decided to discard this data. 

In [37]:
print(df['mill'].unique())

df['mill'].value_counts(normalize=True) 

['Finca El Paraiso' 'Royal Bean Geisha Estate'
 'oklao coffee processing plant' 'La Montana Tarrazu MIll'
 'Finca Santuario' 'Dinámica Café' '野牡丹咖啡' '七彩琉璃咖啡莊園' '亮軒咖啡莊園'
 'GOURMET COFFEE MILL' 'Moledo社\u3000委託精選場' 'Cafetoland' '古峰咖啡莊園'
 'Dry Mill or Hulling Facility' 'La Gaitania' '青葉咖啡莊園'
 'Halo Bariti Cooprative' 'yes' 'El Vergel'
 '仲大叔咖啡莊園Uncle Chung.s Coffee Farm'
 'Machines to peel outer skin of the cherry coffee and coffee is dried manually'
 'El Diamante' '永舜咖啡莊園' 'Dry mill' '鄒築園ZouZhouYuan' '嵩岳咖啡莊園' '他扶芽有機農園'
 'Kona Coffee & Tea' '鄉庭有機農場Siang-Ting Organic Farm'
 '金讚咖啡農莊園(Jinzan Coffee Estate)' 'Lata Agri export' 'dry mill'
 'GUJI COFFEE EXPORT P.L.C' '卓武山咖啡農場' '御香咖啡園 YU SIANG Coffee Estate' 'NKG'
 '花蓮縣秀林鄉特用作物（咖啡）產銷第一班Agriculture Production and Marketing Groups of Hualien Shlin township special crop (coffee) 1st class'
 '乾燥機、脫殼機' '香夾蘭莊園' 'Dry Mill' '愛姬咖啡莊園iGfarm' '青山坪咖啡農場'
 'Beneficio Pastores' 'BENEFICIO LAS AMERICAS'
 'inmobiliaria e inversiones dos mil, s.a.' '六曲窩' '北平山林' 'PT 

mill
Dry Mill                 0.053922
yes                      0.024510
Dry mill                 0.019608
GOURMET COFFEE MILL      0.019608
Agua Caliente            0.014706
                           ...   
名陽園                      0.004902
CupnCare                 0.004902
苗55咖啡莊園                  0.004902
5.2Ha                    0.004902
Beneficio humedo/seco    0.004902
Name: proportion, Length: 162, dtype: float64

In [38]:
from langdetect import detect_langs

def detect_language(text):
    try:
        return detect_langs(text)[0].lang
    except Exception as e:
        return 'unknown'

# detect language of the 'mill' column
df_mill = df[['mill']].copy()
df_mill['mill_lang'] = df_mill['mill'].apply(detect_language)
# if mill_lang != 'en' then replace with 'other'
df_mill.loc[df_mill['mill_lang'] != 'en', 'mill_lang'] = 'other'
print(df_mill['mill_lang'].value_counts(normalize=True))
print(f"\nData in 'mill' column that is not in English: {df_mill['mill_lang'].value_counts(normalize=True)['other']:.2%}")
print(f"Data in 'mill' column that is in English: {df_mill['mill_lang'].value_counts(normalize=True)['en']:.2%}")

mill_lang
other    0.748792
en       0.251208
Name: proportion, dtype: float64

Data in 'mill' column that is not in English: 74.88%
Data in 'mill' column that is in English: 25.12%


In [39]:
# drop 'mill' column
df.drop(columns=['mill'], inplace=True)

<a id="remove_irrevant_columns"></a>
## Remove Irrevant Columns

📍 According to the purpose of this project is to create coffee quality prediction, I will consider that data fields `expiration, certification_body, certification_address, certification_contact` have `low irrevance` to the coffee quality because of these features represent administrative information rather than coffee properties that directly impact quality.

Moreover I consider that `bag_weight, number_of_bags` are also not irrevant since these like information about packaging format and I think it's not directly affect coffee quality.

In [40]:
df.columns

Index(['country_of_origin', 'ico_number', 'company', 'altitude', 'region',
       'number_of_bags', 'bag_weight', 'in_country_partner', 'harvest_year',
       'grading_date', 'owner', 'variety', 'processing_method', 'aroma',
       'flavor', 'aftertaste', 'acidity', 'body', 'balance', 'uniformity',
       'overall', 'total_cup_points', 'moisture_percentage',
       'category_one_defects', 'quakers', 'color', 'category_two_defects',
       'expiration', 'certification_body', 'certification_address',
       'certification_contact'],
      dtype='object')

In [41]:
# found that 'expiration' is clearly related to 'grading_date' since the expiration date is 12 months after the grading date for all data
# so 'expiration' is might be related to a specific coffee quality certification not directly related to the coffee quality itself
period = (pd.to_datetime(df['expiration']) - pd.to_datetime(df['grading_date'])).dt.days
period_months = period / 30
period_months.unique()

array([12.16666667])

In [42]:
# drop expiration, certification_body, certification_address, certification_contact
df.drop(columns=[
    'expiration', 'certification_body', 'certification_address', 'certification_contact',
    'bag_weight', 'number_of_bags',
    ], inplace=True)

In [43]:
df.columns, df.shape

(Index(['country_of_origin', 'ico_number', 'company', 'altitude', 'region',
        'in_country_partner', 'harvest_year', 'grading_date', 'owner',
        'variety', 'processing_method', 'aroma', 'flavor', 'aftertaste',
        'acidity', 'body', 'balance', 'uniformity', 'overall',
        'total_cup_points', 'moisture_percentage', 'category_one_defects',
        'quakers', 'color', 'category_two_defects'],
       dtype='object'),
 (207, 25))

<a id="exploring_data"></a>
# Exploring Data

In [44]:
numeric_df, object_df = get_info(df)

In [45]:
numeric_df

,column,dtype,valid,missing,unique,unique_pct,mean,std,min,25%,50%,75%,max
0,altitude,float64,207,0,77,37.20,1313.25,720.08,139.00,1020.00,1340.00,1600.000,5400.00
1,harvest_year,int64,207,0,5,2.42,2021.43,0.66,2017.00,2021.00,2021.00,2022.000,2023.00
2,aroma,float64,207,0,19,9.18,7.72,0.29,6.50,7.58,7.67,7.920,8.58
3,flavor,float64,207,0,19,9.18,7.74,0.28,6.75,7.58,7.75,7.920,8.50
4,aftertaste,float64,207,0,20,9.66,7.60,0.28,6.67,7.42,7.58,7.750,8.42
5,acidity,float64,207,0,19,9.18,7.69,0.26,6.83,7.50,7.67,7.875,8.58
6,body,float64,207,0,17,8.21,7.64,0.23,6.83,7.50,7.67,7.750,8.25
7,balance,float64,207,0,18,8.70,7.64,0.26,6.67,7.50,7.67,7.790,8.42
8,uniformity,float64,207,0,3,1.45,9.99,0.10,8.67,10.00,10.00,10.000,10.00
9,overall,float64,207,0,21,10.14,7.68,0.31,6.67,7.50,7.67,7.920,8.58


In [46]:
object_df

,column,dtype,valid,missing,unique,unique_pct,top,freq
0,country_of_origin,object,207,0,22,10.63,Taiwan,61
1,ico_number,bool,207,0,2,0.97,False,137
2,company,object,207,0,72,34.78,Taiwan Coffee Laboratory,51
3,region,object,207,0,120,57.97,Chiayi,12
4,in_country_partner,object,207,0,21,10.14,Taiwan Coffee Laboratory 台灣咖啡研究室,83
5,grading_date,object,207,0,75,36.23,2022-11-15,40
6,owner,object,207,0,80,38.65,Taiwan Coffee Laboratory 台灣咖啡研究室,30
7,variety,object,207,0,46,22.22,Caturra,34
8,processing_method,object,207,0,10,4.83,Washed / Wet,127
9,color,object,207,0,12,5.80,green,101


<a id="numerical_data"></a>
## Numerical Data

Outlier

In [47]:
def plot_box(df: pd.DataFrame, column: str=None, boxpoints:str=None):
    """
    This function plots boxplot for the column
    Args:
        df: a dataframe that you want to plot
        column: a specific column that you want to plot
    """
    fig = go.Figure()
    if column:
        fig.add_trace(go.Box(y=df[column], name=column, boxmean=True, boxpoints=boxpoints))
        fig.update_layout(title=f'Distribution of the {column}')
    else:
        numeric_df, _ = get_info(df)
        for i, row in numeric_df.iterrows():
            fig.add_trace(go.Box(y=df[row['column']], name=row['column'], boxmean=True, boxpoints=boxpoints))
        fig.update_layout(title=f'Distribution of the numerical columns')
    fig.show()

def plot_bar(x: list, y: list, xaxis_title: str, yaxis_title: str):
    """
    This function plots bar chart
    Args:
        x: list of x values
        y: list of y values
        xaxis_title: title of x-axis
        yaxis_title: title of y-axis
    """
    # plot bar chart
    fig = go.Figure()
    # bar plot
    fig.add_trace(go.Bar(x=x, y=y))
    fig.update_layout(title=f'{xaxis_title} and {yaxis_title} Distribution', xaxis_title=xaxis_title, yaxis_title=yaxis_title)
    fig.show()

    # correct outliers with IQR method
def correct_outliers(df, column):
    """
    This function corrects outliers using IQR method
    Args:
        df: pd.DataFrame
        column: str
    Returns:
        pd.DataFrame
    """
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df.loc[df[column] < lower_bound, column] = lower_bound
    df.loc[df[column] > upper_bound, column] = upper_bound
    return df

In [48]:
plot_box(df)

1. 'altitude' outlier

'altitude' has extreamly outliers, I will check it with region first. From dataset [discussion](https://www.kaggle.com/datasets/fatihb/coffee-quality-data-cqi/discussion/411169) community discuss that 'altitude' is in meters unit but these outliers are in ft unit so I will change it to meters.
- Santa Rosa 
- ANTIGUA GUATEMALA
- Lam Dong Province

In [49]:
# 'altitude' has extreamly outliers, I will check it with region first
df.query("altitude > 3000")

,country_of_origin,ico_number,company,altitude,region,in_country_partner,harvest_year,grading_date,owner,variety,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,overall,total_cup_points,moisture_percentage,category_one_defects,quakers,color,category_two_defects
47,Guatemala,True,"OLAM AGRO GUATEMALA, S.A.",4700.0,Santa Rosa,Asociacion Nacional Del Café,2022,2023-02-23,Yesica Alejandra Martìnez Vàsquez,Catuai,Washed / Wet,7.67,8.00,7.75,7.92,8.00,7.83,10.0,7.83,85.00,11.3,0,0,green,4
60,Guatemala,True,"VALBROS, S. A.",4895.0,ANTIGUA GUATEMALA,Asociacion Nacional Del Café,2021,2022-05-19,Angelica Paola Citan Lopez,Bourbon,Natural / Dry,7.83,8.00,7.67,7.83,7.75,7.75,10.0,7.83,84.67,10.9,0,3,yellow-green,2
99,Guatemala,True,Asociación Nacional del Cafe,5272.5,ANTIGUA GUATEMALA,Asociacion Nacional Del Café,2021,2022-05-19,Angelica Paola Citan Lopez,Bourbon,Washed / Wet,7.58,7.92,7.50,7.67,7.75,7.67,10.0,7.75,83.83,10.9,0,1,greenish,4
182,Vietnam,False,Brew Baby Coffee Company,5400.0,Lam Dong Province,Firedancer Coffee Consultants,2022,2023-04-10,Rodney Murray,Catimor,Pulped natural / honey,7.50,7.50,7.25,7.42,7.42,7.42,10.0,7.33,81.83,11.2,0,0,green,4


In [50]:
# Convert ft to m
df.loc[df['altitude'] > 3000, 'altitude'] = df.loc[df['altitude'] > 3000, 'altitude'] * 0.3048

In [51]:
# after correct altitude outliers
plot_box(df)

2. 'moisture_percentage' outlier

In [52]:
plot_box(df, 'moisture_percentage')

In [53]:
# correct outliers in 'moisture_percentage'
df = correct_outliers(df, 'moisture_percentage')
plot_box(df, 'moisture_percentage')

Univariate analysis

In [54]:
# plot bar chart for each column representing the distribution of the data
for column in numeric_df['column']:
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=df[column], name=column))
    # add vertical line for the mean
    fig.add_vline(x=df[column].mean(), line_dash="dash", line_color="red", annotation_text=f"Mean: {df[column].mean():.2f}")
    # fig = px.histogram(df, x=column, title=f'Distribution of {column}')
    fig.update_layout(title=f'Distribution of {column}', xaxis_title=column, yaxis_title='Frequency')
    fig.show()

📒 for univariate analysis found that 'harvest_year', 'uniformity' and 'category_one_defects' are extreamly imbalance between the most frequent value and other value (like unique value case). It may not strong enough to be used as feature for 'total_cup_points' prediction. 

Correlation

In [55]:
# correlation matrix of the numerical columns
correlation_matrix = df[numeric_df['column'].values].corr()
fig = go.Figure(data=go.Heatmap(
        z=correlation_matrix,
        x=correlation_matrix.columns,
        y=correlation_matrix.columns,
        colorscale='Blackbody',
        text=correlation_matrix.values,  # The values to display
        texttemplate="%{text:.2f}",  # Format to 2 decimal places
        textfont={"size":5},
        hovertemplate='Correlation: %{text:.3f}<extra></extra>'))
fig.update_layout(title='Correlation Matrix of Numerical Dataframe', width=800, height=800)
fig.show()

In [56]:
# sort the correlation matrix by the target column
correlation_matrix['total_cup_points'].sort_values(ascending=False)

total_cup_points        1.000000
overall                 0.947217
flavor                  0.939124
aftertaste              0.934813
balance                 0.929520
acidity                 0.897057
aroma                   0.868919
body                    0.847216
altitude                0.172389
uniformity              0.003556
category_one_defects   -0.058156
moisture_percentage    -0.065405
harvest_year           -0.127567
category_two_defects   -0.314092
quakers                -0.320307
Name: total_cup_points, dtype: float64

📒 As you can see the correlation between numerical data and our target 'total_cup_point'
- Strong correlate with `overall, flavor , afterta, balance, acidity, aroma, body` since we know there are inputs to compute the real 'total_cup_point' in direct variation
- `category_two_defects, quakers` are also have correlation to 'total_cup_point' but in inverse variation
- As I mention in univariate analysis 'uniformity', 'category_one_defects' and 'harvest_year' have low correlation with 'total_cup_point'.

<a id="categorical_data"></a>
## Categorical Data

In [57]:
object_df

,column,dtype,valid,missing,unique,unique_pct,top,freq
0,country_of_origin,object,207,0,22,10.63,Taiwan,61
1,ico_number,bool,207,0,2,0.97,False,137
2,company,object,207,0,72,34.78,Taiwan Coffee Laboratory,51
3,region,object,207,0,120,57.97,Chiayi,12
4,in_country_partner,object,207,0,21,10.14,Taiwan Coffee Laboratory 台灣咖啡研究室,83
5,grading_date,object,207,0,75,36.23,2022-11-15,40
6,owner,object,207,0,80,38.65,Taiwan Coffee Laboratory 台灣咖啡研究室,30
7,variety,object,207,0,46,22.22,Caturra,34
8,processing_method,object,207,0,10,4.83,Washed / Wet,127
9,color,object,207,0,12,5.80,green,101


In [58]:
def group_classes(df, column, threshold=0.01, name='Other'):
    """
    This function groups classes with a threshold
    Args:
        df: pd.DataFrame
        column: str
        threshold: float
    Returns:
        pd.DataFrame
    """
    value_counts = df[column].value_counts(normalize=True)
    classes = value_counts[value_counts < threshold].index
    df.loc[df[column].isin(classes), column] = name
    return df

def group_classes_with_dict(field, groups):
    for group, members in groups.items():
        if field in members:
            return group
    return 'Other'

1. country_of_origin

In [59]:
# 'country_of_origin'
print(df['country_of_origin'].value_counts(normalize=True))
# rename 'Tanzania, United Republic Of' to 'Tanzania'
df.loc[df['country_of_origin'] == 'Tanzania, United Republic Of', 'country_of_origin'] = 'Tanzania'
# by geographical location
country_groups = {
    'Asia Pacific': ['Taiwan', 'Thailand', 'Vietnam', 'Indonesia', 'Laos', 'Myanmar'],
    'Central America': ['Honduras', 'Costa Rica', 'Guatemala', 'Nicaragua', 'Panama', 'El Salvador'],
    'South America': ['Peru', 'Colombia', 'Brazil'],
    'North America': ['United States (Hawaii)', 'Mexico'],
    'Africa': ['Madagascar', 'Tanzania', 'Uganda', 'Ethiopia', 'Kenya']
}
df['country_of_origin'] = df['country_of_origin'].apply(lambda x: group_classes_with_dict(x, country_groups))
print("\nAfter Grouping:")
print(df['country_of_origin'].value_counts())

country_of_origin
Taiwan                          0.294686
Guatemala                       0.101449
Colombia                        0.091787
Honduras                        0.062802
Thailand                        0.057971
Ethiopia                        0.053140
Brazil                          0.048309
Costa Rica                      0.038647
Nicaragua                       0.033816
El Salvador                     0.033816
Tanzania, United Republic Of    0.028986
United States (Hawaii)          0.024155
Mexico                          0.019324
Peru                            0.019324
Vietnam                         0.019324
Uganda                          0.014493
Indonesia                       0.014493
Laos                            0.014493
Panama                          0.009662
Kenya                           0.009662
Madagascar                      0.004831
Myanmar                         0.004831
Name: proportion, dtype: float64

After Grouping:
country_of_origin
Asia Pacific

2. ico_number

'ico_number is skipped since previously I already transform it to boolean.

3. company

In [60]:
# 'company'
print(df['company'].value_counts())
# group classes with a threshold of 1% avoid Other becomes the majority
df_temp = group_classes(df.copy(), 'company', threshold=0.01)
print("\nAfter Grouping:")
print(df_temp['company'].value_counts())

company
Taiwan Coffee Laboratory        51
Taiwu Coffee Cooperative        25
Coffee Quality Union            15
Doi Tung Development Project     7
Peter Schoenfeld, S.A.           6
                                ..
Taylor Winch Coffee Ltd          1
ECOM COLOMBIA                    1
Exportadora Café California      1
Coffee Quality Institute         1
marubeni                         1
Name: count, Length: 72, dtype: int64

After Grouping:
company
Other                                70
Taiwan Coffee Laboratory             51
Taiwu Coffee Cooperative             25
Coffee Quality Union                 15
Doi Tung Development Project          7
Peter Schoenfeld, S.A.                6
Mercon Honduras                       4
Marubeni Corporation                  4
Mr.Brown Cafe                         4
宸嶧國際有限公司                              3
InterAmerican Coffee                  3
CECA S.A.                             3
Cafe Organico Marcala S.A de C.V.     3
Consejo Salvadoreño del 

4. region

Actually the most frequent classes only representing ~5% but I want to explore more from this point. Since I assume the regions can significantly impact quality of coffee so I convert these regions to location represented in [lat, lon].

In [61]:
# region
print(df['region'].value_counts(normalize=True))

region
Chiayi                         0.057971
新竹縣                            0.053140
苗栗縣                            0.033816
North of Thailand              0.033816
Yunlin                         0.033816
                                 ...   
Addis Ababa                    0.004831
Ka'u district of Big Island    0.004831
玉里鎮Yuli Township               0.004831
Ethiopia                       0.004831
Minas Gerais                   0.004831
Name: proportion, Length: 120, dtype: float64


In [62]:
# read the coordinates from the json file
with open('data/region_lat_lon.json', 'r') as f:
    region_lat_lon = json.load(f)
# print the first 5 items
print(list(region_lat_lon.items())[:5])

[('Piendamo,Cauca', [2.636706, -76.5061489]), ('Chiayi', [23.4518428, 120.2554615]), ('Laos Borofen Plateau', [15.0, 106.0]), ('Los Santos,Tarrazu', [9.65965, -84.02138]), ('Popayan,Cauca', [2.45417, -76.60917])]


In [63]:
lat_lon_df = df.copy()
# map the region to the coordinates
lat_lon_df['region_lat_lon'] = lat_lon_df['region'].map(region_lat_lon)
# split the coordinates into latitude and longitude
lat_lon_df[['region_lat', 'region_lon']] = pd.DataFrame(lat_lon_df['region_lat_lon'].tolist(), index=lat_lon_df.index)
# check the missing values
lat_lon_df[lat_lon_df['region_lat_lon'].isnull()]
# add to df
df['region_lat'] = lat_lon_df['region_lat']
df['region_lon'] = lat_lon_df['region_lon']

In [64]:
fig = px.scatter_geo(df, 
                     lat='region_lat', 
                     lon='region_lon', 
                     color='total_cup_points', 
                     hover_name='region', 
                     size='total_cup_points',
                     projection='natural earth')
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue")
fig.show()

In [65]:
# average total_cup_points by region that in coffee_belt and not in coffee_belt
coffee_belt_lat = [25, -30]
df_tmp = df.copy()
df_tmp['coffee_belt'] = df_tmp.apply(lambda x: "inside coffee belt" if (x['region_lat'] > coffee_belt_lat[1]) and (x['region_lat'] < coffee_belt_lat[0]) else "outside coffee belt", axis=1)

In [66]:
# average total_cup_points by coffee_belt
df_tmp = df_tmp.groupby('coffee_belt')['total_cup_points'].mean().sort_values(ascending=False)
# plot bar chart for country_of_origin
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df_tmp.index, y=df_tmp.values,))
fig.update_layout(title='Coffee Belt Distribution', xaxis_title='Coffee Belt', yaxis_title='Total Cup Points')
fig.show()

5. in_country_partner

In [67]:
print(df['in_country_partner'].value_counts(normalize=True))

a = group_classes(df.copy(), 'in_country_partner', threshold=0.01)
a['in_country_partner'].value_counts(normalize=True)

in_country_partner
Taiwan Coffee Laboratory 台灣咖啡研究室                                     0.400966
Japan Coffee Exchange                                                0.130435
Asociacion Nacional Del Café                                         0.067633
Instituto Hondureño del Café                                         0.048309
FABB Academy of Coffee                                               0.043478
Kenya Coffee Traders Association                                     0.038647
Specialty Coffee Association of Costa Rica                           0.033816
Blossom Valley International宸嶧國際                                     0.028986
METAD Agricultural Development plc                                   0.028986
Salvadoran Coffee Council                                            0.028986
Centro Agroecológico del Café A.C.                                   0.024155
Brazil Specialty Coffee Association                                  0.019324
NKG Quality Service (a division of Bernhard R

in_country_partner
Taiwan Coffee Laboratory 台灣咖啡研究室                                     0.400966
Japan Coffee Exchange                                                0.130435
Asociacion Nacional Del Café                                         0.067633
Instituto Hondureño del Café                                         0.048309
FABB Academy of Coffee                                               0.043478
Kenya Coffee Traders Association                                     0.038647
Specialty Coffee Association of Costa Rica                           0.033816
Salvadoran Coffee Council                                            0.028986
Blossom Valley International宸嶧國際                                     0.028986
METAD Agricultural Development plc                                   0.028986
Centro Agroecológico del Café A.C.                                   0.024155
Brazil Specialty Coffee Association                                  0.019324
Specialty Coffee Association                 

6. owner

In [68]:
# print(df['owner'].unique())
print(df['owner'].value_counts(normalize=True))
# group classes with a threshold
df_temp = group_classes(df.copy(), 'owner', threshold=0.01)
print("\nAfter Grouping:")
print(df_temp['owner'].value_counts(normalize=True))

owner
Taiwan Coffee Laboratory 台灣咖啡研究室     0.144928
Taiwu                                0.120773
Coffee Quality Union                 0.072464
Yesica Alejandra Martìnez Vàsquez    0.038647
Doi Tung Development Project         0.033816
                                       ...   
Sucafina Colombia SAS                0.004831
Marc Dumont                          0.004831
Kao,SIng-jay                         0.004831
Taylor Winch (Coffee) Ltd.           0.004831
SAJONIA ESTATE COFFEE S.A.           0.004831
Name: proportion, Length: 80, dtype: float64

After Grouping:
owner
Other                                                     0.352657
Taiwan Coffee Laboratory 台灣咖啡研究室                          0.144928
Taiwu                                                     0.120773
Coffee Quality Union                                      0.072464
Yesica Alejandra Martìnez Vàsquez                         0.038647
Doi Tung Development Project                              0.033816
Angelica Paola Cit

7. Variety

Reduce the coffee variety to 12 meaningful groups based on:
- Genetic relationships (Bourbon, Typica, Caturra families)
- Disease resistance characteristics (Catimor Group)
- Regional significance (Ethiopian, Indonesian varieties)
- Specialty status (Gesha group)

In [69]:
variety_groups = {
    'Bourbon Family': [
        'Bourbon', 'Yellow Bourbon', 'Red Bourbon', 'Bourbon Sidra', 
        'Red Bourbon,Caturra'  # Hybrids with Bourbon go here
    ],
    
    'Caturra Family': [
        'Caturra', 'Caturra-Catuai', 'Caturra,Colombia,Castillo',
        'Castillo,Caturra,Bourbon'
    ],
    
    'Catuai Family': [
        'Catuai', 'Yellow Catuai', 'Catuai and Mundo Novo'
    ],
    
    'Typica Family': [
        'Typica', 'Typica + SL34', 'Typica Gesha', 'Maragogype'  # Maragogype is a Typica mutation
    ],
    
    'Gesha/Specialty': [
        'Gesha', 'Typica Gesha', 'Sl34+Gesha', 'Pacamara'
    ],
    
    'SL Varieties': [
        'SL28', 'SL34', 'SL14', 'SL28,SL34,Ruiru11'
    ],
    
    'Catimor Group': [
        'Catimor', 'Sarchimor', 'Catucai', 'Parainema', 'Lempira'  # Rust-resistant varieties
    ],
    
    'Castillo Group': [
        'Castillo', 'Castillo Paraguaycito', 'Castillo and Colombia blend'
    ],
    
    'Regional Varieties': [
        'Ethiopian Heirlooms', 'Java', 'Gayo', 'Wolishalo,Kurume,Dega',
        'Jember,TIM-TIM,Ateng', 'Santander', 'Catrenic'
    ],
    
    'Mundo Novo Group': [
        'Mundo Novo', 'Catuai and Mundo Novo'
    ],
    
    'Mixed Varieties': [
        'BOURBON, CATURRA Y CATIMOR', 'Typica Bourbon Caturra Catimor',
        'Bourbon, Catimor, Caturra, Typica', 'Catimor,Catuai,Caturra,Bourbon',
        'MARSELLESA, CATUAI, CATURRA & MARSELLESA, ANACAFE 14, CATUAI'
    ],
    
    'Other': [
        'SHG', 'Pacas'  # SHG is actually a grade, not a variety
    ]
}

In [70]:
# variety_group
df['variety'] = df['variety'].apply(lambda x: group_classes_with_dict(x, variety_groups))
df['variety'].value_counts()

variety
Caturra Family        37
Typica Family         35
Gesha/Specialty       30
Bourbon Family        23
Catimor Group         18
Regional Varieties    17
Catuai Family         15
SL Varieties          14
Mundo Novo Group       5
Mixed Varieties        5
Castillo Group         4
Other                  4
Name: count, dtype: int64

8. processing_method

Reduce the processing method to 5 main groups based on:
[Link](https://www.wearecoffeeco.com/blogs/news/coffee-processing-methods?srsltid=AfmBOoqP2CVT_vR_Q9BCDLjoQehyHzv6QvV7DFF8jUiVa8CVSvk6PUXs)


In [71]:
# Define processing method groups
processing_groups = {
    'Washed/Wet Process': [
        'Washed / Wet', 
        'SEMI-LAVADO'  # Spanish for semi-washed
    ],
    
    'Natural/Dry Process': [
        'Natural / Dry'
    ],
    
    'Honey/Pulped Natural Process': [
        'Pulped natural / honey',
        'Honey,Mossto',
    ],
    
    'Wet Hulling Process': [
        'Wet Hulling',  # Indonesian method
        'Semi Washed'
    ],
    
    'Experimental/Anaerobic Process': [
        'Double Anaerobic Washed',
        'Double Carbonic Maceration / Natural',
        'Anaerobico 1000h'
    ]
}

In [72]:
# processing_method
df['processing_method'] = df['processing_method'].apply(lambda x: group_classes_with_dict(x, processing_groups))
print(df['processing_method'].value_counts(normalize=True))
# group classes with a threshold
df = group_classes(df.copy(), 'processing_method', threshold=0.05)
print("\nAfter Grouping:")
print(df['processing_method'].value_counts(normalize=True))

processing_method
Washed/Wet Process                0.618357
Natural/Dry Process               0.222222
Honey/Pulped Natural Process      0.135266
Experimental/Anaerobic Process    0.014493
Wet Hulling Process               0.009662
Name: proportion, dtype: float64

After Grouping:
processing_method
Washed/Wet Process              0.618357
Natural/Dry Process             0.222222
Honey/Pulped Natural Process    0.135266
Other                           0.024155
Name: proportion, dtype: float64


9. color

In [73]:
# Dictionary for color grouping
color_groups = {
    'Green': ['green', 'greenish'],
    'Blue-Green': ['blue-green', 'bluish-green'],
    'Yellow-Green': ['yellow-green', 'yellow green', 'yellow- green', 'yellow-green', 'yello-green'],
    'Yellow': ['yellowish', 'pale yellow'],
    'Brown/Brownish-Green': ['brownish', 'brownish-green', 'brownish-green', 'browish-green'],
    'Other': []  # Default category for any colors not matched above
}

In [74]:
# color
df['color'] = df['color'].apply(lambda x: group_classes_with_dict(x, color_groups))
print(df['color'].value_counts(normalize=True))

color
Green                   0.661836
Blue-Green              0.159420
Yellow-Green            0.082126
Yellow                  0.048309
Brown/Brownish-Green    0.048309
Name: proportion, dtype: float64


Univariate analysis

In [75]:
# plot bar chart for each column representing the distribution of the data
for column in object_df['column']:
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=df[column], name=column))
    # fig = px.histogram(df, x=column, title=f'Distribution of {column}')
    fig.update_layout(title=f'Distribution of {column}', xaxis_title=column, yaxis_title='Frequency')
    fig.show()

Plot between categorical column and 'total_cup_points'

In [76]:
for column in object_df['column']:
    fig = px.box(df, x=column, y='total_cup_points', points="all", title=f'Total Cup Points by {column}')
    fig.update_layout(xaxis_title=column, yaxis_title='Total Cup Points')
    fig.show()
# fig = px.box(df, x="color", y="total_cup_points", points="all", title="Total Cup Points by Color")
# fig.update_layout(xaxis_title='Color', yaxis_title='Total Cup Points', width=500)
# fig.show()

<a id="model_prediction"></a>
# Model Prediction

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [78]:
# select numerical columns from correlation matrix
print(correlation_matrix['total_cup_points'].sort_values(ascending=False))

numeric_feature_names = ['overall', 'flavor', 'afterta', 'balance', 'acidity', 'aroma', 'body', 'category_two_defects', 'quakers']

total_cup_points        1.000000
overall                 0.947217
flavor                  0.939124
aftertaste              0.934813
balance                 0.929520
acidity                 0.897057
aroma                   0.868919
body                    0.847216
altitude                0.172389
uniformity              0.003556
category_one_defects   -0.058156
moisture_percentage    -0.065405
harvest_year           -0.127567
category_two_defects   -0.314092
quakers                -0.320307
Name: total_cup_points, dtype: float64


Model

In [ ]:
# train
# Assuming df is your dataframe and 'target' is your continuous target variable
X = df.drop('total_cup_points', axis=1)
y = df['total_cup_points']

# First, split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify numeric and categorical columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# Create preprocessing pipelines for both numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Replace LabelEncoder with OrdinalEncoder
categorical_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# Results dictionary
results = {}
feature_importances = {}

# Loop through each model
for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Create pipeline with current model
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('selector', SelectKBest(score_func=f_regression, k=10)),
        ('model', model)
    ])
    
    # Cross-validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores)
    
    # Train on full training set
    pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Store results
    results[name] = {
        'CV RMSE': cv_rmse.mean(),
        'CV RMSE Std': cv_rmse.std(),
        'Train RMSE': train_rmse,
        'Test RMSE': test_rmse,
        'Test MAE': test_mae,
        'Test R²': test_r2
    }
    
    print(f"  CV RMSE: {cv_rmse.mean():.4f} (±{cv_rmse.std():.4f})")
    print(f"  Test RMSE: {test_rmse:.4f}")
    print(f"  Test R²: {test_r2:.4f}")
    
    # Get selected features
    preprocessor = pipeline.named_steps['preprocessor']
    selector = pipeline.named_steps['selector']
    
    # Combine feature names (original names for both numeric and categorical)
    all_feature_names = np.array(list(numeric_features) + list(categorical_features))
    
    # Get mask of selected features
    selected_mask = selector.get_support()
    selected_features = all_feature_names[selected_mask]
    
    print(f"  Selected features: {', '.join(selected_features)}")
    
    # For tree-based models, get feature importance
    if hasattr(model, 'feature_importances_'):
        # Get feature importances from the model
        importances = pipeline.named_steps['model'].feature_importances_
        
        # Create DataFrame with feature names and importances
        feature_imp_df = pd.DataFrame({
            'Feature': selected_features,
            'Importance': importances
        }).sort_values('Importance', ascending=False)
        
        feature_importances[name] = feature_imp_df

# Convert results to DataFrame for easy comparison
results_df = pd.DataFrame(results).T
print("\nModel Comparison:")
print(results_df)

# Create feature importance plots for tree-based models using Plotly
if feature_importances:
    # Create subplots for feature importance
    fig_importance = make_subplots(
        rows=3, cols=2,
        subplot_titles=list(feature_importances.keys()),
        vertical_spacing=0.1
    )
    
    # Add feature importance bars for each model
    row, col = 1, 1
    for name, imp_df in feature_importances.items():
        fig_importance.add_trace(
            go.Bar(
                x=imp_df['Importance'],
                y=imp_df['Feature'],
                orientation='h',
                name=name
            ),
            row=row, col=col
        )
        
        # Update layout for each subplot
        fig_importance.update_xaxes(title_text="Importance", row=row, col=col)
        fig_importance.update_yaxes(title_text="Feature", row=row, col=col)
        
        # Move to next subplot position
        col += 1
        if col > 2:
            col = 1
            row += 1
    
    # Update overall layout
    fig_importance.update_layout(
        height=1000,
        width=1200,
        title_text="Feature Importance by Model",
        showlegend=False
    )
    
    fig_importance.show()

# Plot model comparison - RMSE
fig_rmse = go.Figure()
for model_name in results_df.index:
    fig_rmse.add_trace(go.Bar(
        x=[model_name],
        y=[results_df.loc[model_name, 'CV RMSE']],
        name='CV RMSE',
        error_y=dict(
            type='data',
            array=[results_df.loc[model_name, 'CV RMSE Std']],
            visible=True
        )
    ))
    fig_rmse.add_trace(go.Bar(
        x=[model_name],
        y=[results_df.loc[model_name, 'Test RMSE']],
        name='Test RMSE'
    ))

fig_rmse.update_layout(
    title='Model Comparison - RMSE',
    xaxis_title='Model',
    yaxis_title='RMSE',
    barmode='group',
    height=600,
    width=1000
)
fig_rmse.show()

# Plot R² comparison
fig_r2 = px.bar(
    results_df.reset_index(),
    x='index',
    y='Test R²',
    title='Model Comparison - R²',
    labels={'index': 'Model', 'Test R²': 'R² Score'}
)

fig_r2.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=len(results_df)-0.5,
    y1=0,
    line=dict(color="red", width=2, dash="dash")
)

fig_r2.add_shape(
    type="line",
    x0=-0.5,
    y0=1,
    x1=len(results_df)-0.5,
    y1=1,
    line=dict(color="green", width=2, dash="dash")
)

fig_r2.update_layout(height=600, width=1000)
fig_r2.show()

# Find best model
best_model = results_df['Test R²'].idxmax()
print(f"\nBest model: {best_model} with R² = {results_df.loc[best_model, 'Test R²']:.4f}")

# Retrain best model
best_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(score_func=f_regression, k=10)),
    ('model', models[best_model])
])
best_pipeline.fit(X_train, y_train)
y_test_pred = best_pipeline.predict(X_test)

# Plot actual vs predicted
fig_pred = px.scatter(
    x=y_test,
    y=y_test_pred,
    labels={"x": "Actual", "y": "Predicted"},
    title=f"{best_model} - Actual vs Predicted"
)

# Add perfect prediction line
fig_pred.add_shape(
    type="line",
    x0=y.min(),
    y0=y.min(),
    x1=y.max(),
    y1=y.max(),
    line=dict(color="black", width=2, dash="dash")
)

fig_pred.update_layout(height=600, width=800)
fig_pred.show()

# Plot residuals
residuals = y_test - y_test_pred
fig_resid = px.scatter(
    x=y_test_pred,
    y=residuals,
    labels={"x": "Predicted", "y": "Residuals"},
    title=f"{best_model} - Residuals"
)

# Add zero line
fig_resid.add_shape(
    type="line",
    x0=min(y_test_pred),
    y0=0,
    x1=max(y_test_pred),
    y1=0,
    line=dict(color="red", width=2)
)

fig_resid.update_layout(height=600, width=800)
fig_resid.show()


Training Linear Regression...
  CV RMSE: 0.0830 (±0.0890)
  Test RMSE: 0.0411
  Test R²: 0.9993
  Selected features: aroma, flavor, aftertaste, acidity, body, balance, overall, quakers, category_two_defects, country_of_origin

Training Ridge Regression...
  CV RMSE: 0.0813 (±0.0899)
  Test RMSE: 0.0380
  Test R²: 0.9994
  Selected features: aroma, flavor, aftertaste, acidity, body, balance, overall, quakers, category_two_defects, country_of_origin

Training Lasso Regression...
  CV RMSE: 1.1331 (±0.1876)
  Test RMSE: 1.0111
  Test R²: 0.5905
  Selected features: aroma, flavor, aftertaste, acidity, body, balance, overall, quakers, category_two_defects, country_of_origin

Training Decision Tree...
  CV RMSE: 0.5236 (±0.0899)
  Test RMSE: 0.4461
  Test R²: 0.9203
  Selected features: aroma, flavor, aftertaste, acidity, body, balance, overall, quakers, category_two_defects, country_of_origin

Training Random Forest...
  CV RMSE: 0.3499 (±0.1035)
  Test RMSE: 0.3425
  Test R²: 0.9530
  Sel


Best model: Ridge Regression with R² = 0.9994


<a id="conclusion"></a>
# Conclusion

🚀 Task 4 answer:
With dataset consisting of ~200 rows and mixed type of data. I try simple model first Linear Regression and Decision Tree. Then try ensemble methods like Random Forest Gradient Boosting to compare their performance.
6 models are seleceted to compared the coffee quality prediction in this project consisting of:
- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree
- Random Forest
- Gradient Boosting
And use cross-validation is applied to evaluate each model performance. 
1. The result found that the best performance is `Ridge Regression` selected features: aroma, flavor, aftertaste, acidity, body, balance, overall, quakers, category_two_defects, country_of_origin with Test R²: 0.9993
2. `Linear Regression` is also perform a good result with the same selected features as 'Ridge Regression'. It performs with Test R²: 0.9994
